In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# If the icetemperature library is not in your PYTHONPATH, you will not be able to load those functions
# Check and update here if necessary
import sys
cs_dir = '../'
if cs_dir not in sys.path:
    sys.path.append(cs_dir)

from icetemperature.lib.model import ice_temperature
from icetemperature.lib.analytical_functions import conductivity, heat_capacity
from icetemperature.lib.constants import constants
const = constants()

from scipy.interpolate import interp1d

In [ ]:
# Numbers as input to forward model
z_data,T_data,C_data = np.transpose(np.load('../data/compiled_insitu_icetemp.npy'))
model_accumulation = np.load('../data/paleoAccumulation.npy')
model_airTemp = np.load('../data/paleoAirTemperature.npy')
ts = model_accumulation[0]
adot = model_accumulation[1]
Tsurf = model_airTemp[1]

data = np.genfromtxt('../data/SP19_Depth_Age.csv',skip_header=1,delimiter=',')
z_rho = data[:,0]
rho_data = data[:,5]

H = 2875.
z_data[-1] = -H
T_data[-1] = const.rho*const.g*const.beta*H

In [ ]:
plt.figure(figsize=(6,6))

ax1 = plt.subplot(211)
plt.tick_params(labelbottom=False)
ax1.plot((ts-np.max(ts))/1000.,Tsurf,'k')
plt.ylabel('Temperature ($^\circ$C)')

ax2 = plt.subplot(212)
ax2.plot((ts-np.max(ts))/1000.,adot,'k')
plt.ylabel('Accumulation (m/yr)')
plt.xlabel('kyr');

In [ ]:
m = ice_temperature()
m.ts = ts*const.spy
m.adot = adot/const.spy
m.Ts = Tsurf
m.H = H

m.qgeo = .06
m.p = 1000.
m.Udef = 0.0/const.spy
m.Uslide = 9.3/const.spy

m.flags.append('weertman_vel')
m.dTs = 7.5e-6

m.flags.append('temp-dependent')
m.initial_conditions()
rho_interpolator = interp1d(m.H-z_rho,rho_data,fill_value='extrapolate')
m.rho = 1000.*rho_interpolator(m.z)
m.k = conductivity(m.T.copy(),m.rho)
m.Cp = heat_capacity(m.T.copy(),m.rho)

m.source_terms()
m.stencil()
m.run_to_steady_state()

m.flags.append('save_all')

m.run()

In [ ]:
plt.figure(figsize=(4,4))
plt.plot(T_data,z_data,'k.')
plt.plot(m.T,m.z-m.H,c='indianred')
plt.ylabel('Depth (m)')
plt.xlabel('Temperature ($^\circ$C)')

plt.tight_layout()